In [1]:
from __future__ import division
import ipywidgets as ipw
from bs4 import BeautifulSoup 
import requests
import html5lib
import re
from urllib.parse import urlparse
import pandas as pd

def on_click(btn):
            
    url = output1.value

    parse_object = urlparse(url)
    parse_object.netloc

    r = requests.get(url) 
    data = r.text
    soup = BeautifulSoup(data)

    val = []
    title=[]

    table = soup.find("table", { "class" : "table table-striped" })

    table_body = table.find('tbody')
    rows = table_body.find_all('tr')

    for row in rows:
        cells = row.findAll("td")
        heading = row.findAll("th")
        column=[]

        for ele in cells:
            column.extend(abc.text for abc in ele.findAll("h5"))
            try:
                column.extend(abc.find('a').get('href') for abc in ele.findAll("h5"))
            except:
                next
            column.extend(abc.text for abc in ele.findAll("p"))

        for ele in heading:
            title.extend(str(abc.text) for abc in ele.findAll("h4"))

        if column:
            val.append(column)

    title.insert(2,"Link")
    df=pd.DataFrame(val,columns=title)

    if btn.description == "Ok":
        try:                  
            print (df)
        except:
            output1.value= "ERROR"

    elif btn.description == "Clear":
        output1.value = ""


    elif btn.description == "Generate Excel":
        try:
            df.to_excel("output.xlsx")
        except:
            output1.value= "ERROR"       
    else:
            output1.value= "ERROR"

def mk_btn(description):
    btn = ipw.Button(description=description, button_style='success',layout=ipw.Layout(width='100%',align_items='center',
            justify_items='center',grid_area=description))
    btn.on_click(on_click)
    return btn

output1 = ipw.Text(description="Enter URL",layout=ipw.Layout(width="90%"),  disabled=False)

button1 = ipw.HBox([mk_btn(d) for d in ('Ok','Clear','Generate Excel')], 
                    layout=ipw.Layout(width='70%',align_items='center',justify_items='center'))
ipw.VBox([output1,button1])